In [7]:
import os
import pickle
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (BatchNormalization, ConvLSTM2D, Dense,
                                     Input, LeakyReLU, Conv3D)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from_generator = tf.data.Dataset.from_generator

from DataPrep import DataPrep

In [8]:

# TODO: add variables for frame height and width
# def input_fn(filepath, batch_size=10, segment_size=5):

filepath = 'data/train_sample_videos'
segment_size = 5
datapath = os.path.join(filepath, 'metadata.json')
data = pd.read_json(os.path.join(datapath)).T
files = [os.path.join(filepath, f) for f in data.index]
labels = data.label.values
x_train, x_test, y_train, y_test = train_test_split(
    files, labels, test_size=0.2)
class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
for k, v in zip(np.unique(y_train), class_weights):
    print(k, v)
y_train = list(map(lambda x: 0 if x == 'REAL' else 1, y_train))
y_test = list(map(lambda x: 0 if x == 'REAL' else 1, y_test))
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
print(len(x_train), len(y_train), len(x_test), len(y_test))

FAKE 0.6153846153846154
REAL 2.6666666666666665
320 320 80 80


In [9]:

# @tf.function
def input_fn(files, labels, segment_size=5, batch_size=1):
    def dataGenerator():
        for f, label in zip(files, labels):
            dp = DataPrep(segment_size=segment_size)
            frames = dp.prepFullFrames(filepath=f)
            flows = dp.getOpticalFlows()
            yield {'rgb_input': frames, 'flow_input': flows}, label
    dataset = from_generator(
        dataGenerator,
        output_types=(
            {
                "rgb_input": tf.int8,
                "flow_input": tf.float32
            },
            tf.int8),
        output_shapes=(
            {
                "rgb_input": (segment_size, 256, 256, 3),
                "flow_input": (segment_size - 1, 256, 256, 2)
            },
            (2,))
    )
    dataset = dataset.batch(batch_size)
    return dataset

In [10]:
batch_size = 10
train_data = input_fn(x_train, y_train, batch_size=batch_size)
test_data = input_fn(x_test, y_test, batch_size=batch_size)


In [11]:
class InputStream(tf.keras.Model):
    def __init__(self, kernel_size, filters, name):
        super().__init__(name=name)
        self.convLstm1 = ConvLSTM2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=1,
            padding='same',
            data_format='channels_last',
            return_sequences=True,
            dropout=0.5
        )
        self.convLstm2 = ConvLSTM2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=1,
            padding='same',
            data_format='channels_last',
            return_sequences=False,
            dropout=0.5
        )
        self.bn = BatchNormalization()
        self.act = LeakyReLU()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = Dense(512)
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.out_layer = Dense(2)

    # TODO: specify different behavior for training, i.e. dropout only when
    # training
    def call(self, input_tensor, training=False):
        x = self.convLstm1(input_tensor)
        x = self.bn(x)
        x = self.convLstm1(x)
        x = self.bn(x)
        x = self.convLstm2(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.act(x)
        x = self.dense(x)
        x = self.act(x)
        x = self.dense(x)
        x = self.act(x)
        x = self.dropout(x)
        return self.out_layer(x)



In [12]:
rgb_stream = InputStream(3, 32, 'rgb_stream')
flow_stream = InputStream(3, 32, 'flow_stream')
rgb_input = tf.keras.Input(shape=(5, 256, 256, 3), name='rgb_input')
flow_input = tf.keras.Input(shape=(4, 256, 256, 2), name='flow_input')
rgb = rgb_stream(rgb_input)
flow = flow_stream(flow_input)
final_average = tf.keras.layers.average([rgb, flow])
x = tf.keras.layers.Flatten()(final_average)
final_output = Dense(2, activation='softmax', name='final_output')(x)
model = Model(
    inputs={ "rgb_input": rgb_input, "flow_input": flow_input},
    outputs=final_output,
    name='my_model'
)

ValueError: in converted code:

    <ipython-input-11-1ac709aa669d>:34 call  *
        x = self.convLstm1(x)
    /root/anaconda/envs/deepfake/lib/python3.7/site-packages/tensorflow_core/python/keras/layers/convolutional_recurrent.py:299 __call__
        return super(ConvRNN2D, self).__call__(inputs, **kwargs)
    /root/anaconda/envs/deepfake/lib/python3.7/site-packages/tensorflow_core/python/keras/layers/recurrent.py:623 __call__
        return super(RNN, self).__call__(inputs, **kwargs)
    /root/anaconda/envs/deepfake/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:812 __call__
        self.name)
    /root/anaconda/envs/deepfake/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/input_spec.py:224 assert_input_compatibility
        ', found shape=' + str(shape))

    ValueError: Input 0 is incompatible with layer conv_lst_m2d_2: expected shape=(None, None, 256, 256, 3), found shape=[None, 5, 256, 256, 32]


In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_layer_names=True,
)

In [ ]:
opt = tf.keras.optimizers.Adam()
model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc'])
model.fit(
    train_data,
    epochs=25,
    verbose=1,
    class_weight=class_weights
)

In [ ]:
model.evaluate(
    test_data,
#     class_weight=class_weights
)


In [ ]:
# DEBUGGING
dataset = input_fn(filepath, batch_size=1)
start = time.time()
for i, x in enumerate(dataset):
    print(i)
    data_dict, label = x
    for k, v in data_dict.items():
        print(k, v.numpy().shape)
    print(time.time() - start)


In [ ]:
filepath = 'data/train_sample_videos'
segment_size = 5
datapath = os.path.join(filepath, 'metadata.json')
data = pd.read_json(os.path.join(datapath)).T
files = [os.path.join(filepath, f) for f in data.index]
labels = data.label.apply(lambda x: 0 if x == 'REAL' else 1)
labels = to_categorical(labels, num_classes=2)
for i, (f, label) in enumerate(zip(files, labels)):
    try:
        dp = DataPrep(segment_size=segment_size)
        frames = dp.prepFullFrames(filepath=f)
        flows = dp.getOpticalFlows()
    except Exception as e:
        print(i, f, label)
        print(e)
        break
print('everything is working now')


In [ ]:
f = 'data/train_sample_videos/adhsbajydo.mp4'
dp = DataPrep(segment_size=segment_size)
frames = dp.prepFullFrames(filepath=f)
